# Visualization Demo
Multi-SKU visualization demo (zone overlay, comparison chart, dashboard, custom style).

## 1. Setup
Use VIS_TEST SKU and VIS_* sample images.

In [ ]:
from pathlib import Path
import sys
import time
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt

plt.style.use('seaborn-v0_8')

PROJECT_ROOT = Path.cwd().resolve()
if PROJECT_ROOT.name == 'notebooks':
    PROJECT_ROOT = PROJECT_ROOT.parent
sys.path.insert(0, str(PROJECT_ROOT))

from src.pipeline import InspectionPipeline
from src.visualizer import InspectionVisualizer, VisualizerConfig
from src.utils.file_io import read_json

DATA_DIR = PROJECT_ROOT / 'data' / 'raw_images'
SKU_CODE = 'VIS_TEST'
SKU_PATH = PROJECT_ROOT / 'config' / 'sku_db' / f'{SKU_CODE}.json'

sku_config = read_json(SKU_PATH)
pipeline = InspectionPipeline(sku_config)
visualizer = InspectionVisualizer(VisualizerConfig())

print(f'root: {PROJECT_ROOT}')
print(f'SKU: {SKU_CODE}')


def run_inspection(image_path: Path, sku_code: str = SKU_CODE):
    """Run pipeline step-by-step and return intermediates for visualization."""
    image = pipeline.image_loader.load_from_file(image_path)
    processed = pipeline.image_loader.preprocess(image)
    detection = pipeline.lens_detector.detect(processed)
    profile = pipeline.radial_profiler.extract_profile(processed, detection)
    zones = pipeline.zone_segmenter.segment(profile)
    inspection_result = pipeline.color_evaluator.evaluate(zones, sku_code, sku_config)

    inspection_result.image_path = str(image_path)
    inspection_result.lens_detection = detection
    inspection_result.zones = zones

    return {
        'image_path': image_path,
        'processed_image': processed,
        'detection': detection,
        'zones': zones,
        'result': inspection_result,
    }


def to_rgb(img: np.ndarray) -> np.ndarray:
    return cv2.cvtColor(img, cv2.COLOR_BGR2RGB) if img is not None else img


## 2. Zone overlay (OK vs NG)

In [ ]:
ok_ctx = run_inspection(DATA_DIR / 'VIS_OK_001.jpg')
ng_ctx = run_inspection(DATA_DIR / 'VIS_NG_001.jpg')

ok_result = ok_ctx['result']
ng_result = ng_ctx['result']

ok_overlay = visualizer.visualize_zone_overlay(
    ok_ctx['processed_image'],
    ok_ctx['detection'],
    ok_ctx['zones'],
    ok_result,
    show_result=True
)

ng_overlay = visualizer.visualize_zone_overlay(
    ng_ctx['processed_image'],
    ng_ctx['detection'],
    ng_ctx['zones'],
    ng_result,
    show_result=True
)

fig, axes = plt.subplots(1, 2, figsize=(14, 7))
axes[0].imshow(to_rgb(ok_overlay))
axes[0].set_title(f"OK sample\nDE={ok_result.overall_delta_e:.2f}, conf={ok_result.confidence:.2f}")
axes[0].axis('off')

axes[1].imshow(to_rgb(ng_overlay))
axes[1].set_title(f"NG sample\nDE={ng_result.overall_delta_e:.2f}, conf={ng_result.confidence:.2f}")
axes[1].axis('off')

plt.suptitle('Zone overlay: OK vs NG', fontsize=15, fontweight='bold')
plt.tight_layout()
plt.show()


## 3. Comparison chart (measured vs target)

In [ ]:
ok_fig = visualizer.visualize_comparison(ok_ctx['zones'], ok_result)
ng_fig = visualizer.visualize_comparison(ng_ctx['zones'], ng_result)
plt.show()


## 4. Multi-image overlay

In [ ]:
vis_images = sorted((DATA_DIR).glob('VIS_*\.jpg'))
print(f"found VIS_* images: {len(vis_images)}")
for p in vis_images:
    print(f"  - {p.name}")

all_ctx = [run_inspection(p) for p in vis_images[:6]]
all_results = [c['result'] for c in all_ctx]

fig, axes = plt.subplots(2, 3, figsize=(18, 12))
axes = axes.flatten()

for i, ctx in enumerate(all_ctx[:6]):
    overlay = visualizer.visualize_zone_overlay(
        ctx['processed_image'],
        ctx['detection'],
        ctx['zones'],
        ctx['result'],
        show_result=True
    )
    axes[i].imshow(to_rgb(overlay))
    axes[i].set_title(f"{ctx['image_path'].name}\n{ctx['result'].judgment}, DE={ctx['result'].overall_delta_e:.2f}",
                     fontsize=11, fontweight='bold')
    axes[i].axis('off')

plt.suptitle('Multi-image zone overlay', fontsize=16, fontweight='bold')
plt.tight_layout()
plt.show()


## 5. Dashboard (batch summary)

In [ ]:
dashboard_fig = visualizer.visualize_dashboard(all_results)
plt.show()


## 6. Save outputs

In [ ]:
output_dir = PROJECT_ROOT / 'results' / 'visualization_demo'
output_dir.mkdir(parents=True, exist_ok=True)

visualizer.save_visualization(ok_overlay, output_dir / 'ok_overlay.png')
visualizer.save_visualization(ng_overlay, output_dir / 'ng_overlay.png')
visualizer.save_visualization(ok_fig, output_dir / 'ok_comparison.png')
visualizer.save_visualization(ng_fig, output_dir / 'ng_comparison.png')
visualizer.save_visualization(dashboard_fig, output_dir / 'dashboard.png')

print(f"saved visualizations to {output_dir}")


## 7. Custom style comparison

In [ ]:
custom_config = VisualizerConfig(
    zone_line_thickness=4,
    zone_color_ok=(0, 200, 0),
    zone_color_ng=(0, 0, 255),
    zone_label_font_scale=0.9,
    judgment_banner_height=80
)
custom_visualizer = InspectionVisualizer(custom_config)

custom_overlay = custom_visualizer.visualize_zone_overlay(
    ok_ctx['processed_image'],
    ok_ctx['detection'],
    ok_ctx['zones'],
    ok_result,
    show_result=True
)

fig, axes = plt.subplots(1, 2, figsize=(16, 8))
axes[0].imshow(to_rgb(ok_overlay))
axes[0].set_title('Default config', fontsize=14, fontweight='bold')
axes[0].axis('off')

axes[1].imshow(to_rgb(custom_overlay))
axes[1].set_title('Custom config', fontsize=14, fontweight='bold')
axes[1].axis('off')

plt.suptitle('Visualizer style comparison', fontsize=16, fontweight='bold')
plt.tight_layout()
plt.show()


## Summary
- Zone overlay, comparison chart, dashboard for VIS_TEST samples
- Processed 6 VIS_* images (OK/NG) with DE stats
- Included custom style example